In [ ]:
import numpy as np
from arch.bootstrap import StationaryBootstrap, IIDBootstrap
from arch.__future__ import reindexing
from arch.utility.exceptions import arch_warning
from arch.utility.timeseries import add_trend
import matplotlib.pyplot as plt
from scipy.stats import norm
from arch import arch_model

# Сгенерируем случайные возвраты
np.random.seed(42)
returns = np.random.normal(0, 0.01, 500)  # 500 торговых дней

# Рассчитаем VaR на уровне 95%
alpha = 0.05
VaR = np.percentile(returns, 100 * alpha)

# Определяем превышения
exceedances = returns < VaR

# Kupiec's Proportion of Failures (POF) Test
from arch.__future__.volatility import backtesting

kupiec_test = backtesting.kupiec_pof(exceedances, alpha)

# Christoffersen's Test for Independence of Failures
christoffersen_test = backtesting.cav(exceedances)

print("Kupiec's POF Test:")
print(f"LR statistic: {kupiec_test[0]}, p-value: {kupiec_test[1]}")

print("\nChristoffersen's Test for Independence:")
print(f"LR statistic: {christoffersen_test[0]}, p-value: {christoffersen_test[1]}")

# Hurlin and Tokpavi's Test
# Так как прямой реализации нет, мы моделируем анализ последовательности превышений
def calculate_sequential_test(exceedances):
    seq_exceedances = np.diff(exceedances.astype(int)) == 1
    count_seq_exceedances = np.sum(seq_exceedances)
    expected = np.sum(exceedances) * (1 - alpha)
    test_stat = (count_seq_exceedances - expected)**2 / expected
    return test_stat

ht_test_stat = calculate_sequential_test(exceedances)
print("\nHurlin and Tokpavi's Sequential Test Statistic:")
print(f"Test Statistic: {ht_test_stat}")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.hist(returns, bins=30, alpha=0.75)
plt.axvline(VaR, color='r', linestyle='dashed', linewidth=2)
plt.title('Histogram of Returns and VaR (95% Confidence Level)')
plt.xlabel('Returns')
plt.ylabel('Frequency')
plt.show()